from https://uoregonctn.atlassian.net/wiki/spaces/DE/pages/1730019339/TESQ-E+and+measures+of+response+inhibition+and+suppression

Building the measure: Dissimilarity measure of CorrectStop vs. CorrectGo within subject. Repeat for each subject and we have dissimilarity measures across the group.

Ultimate research question: does this better predict FFQ?

In [40]:
from nltools import Brain_Data
from scipy.stats import ttest_ind

In [2]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *
import gc
import nibabel as nib

nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
train_test_markers_filepath = ml_data_folderpath + "/train_test_markers_20210601T183243.csv"

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")

script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'



python initialized for apply_loocv_and_save
4


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [3]:
brain_data_filepath = ml_data_folderpath + "/SST/Brain_Data_betaseries_15subs_correct_cond.pkl"

In [4]:
bd=pickle.load(open(brain_data_filepath,'rb'))
bd.X

,onset,duration,trial_type,subject,wave,beta
256,0.00000,2.25834,correct-stop,DEV005,wave1,beta_0001.nii
258,2.75834,0.40082,correct-go,DEV005,wave1,beta_0003.nii
260,5.51390,0.66191,correct-go,DEV005,wave1,beta_0005.nii
264,12.52780,0.51712,correct-go,DEV005,wave1,beta_0009.nii
266,15.90975,0.39906,correct-go,DEV005,wave1,beta_0011.nii
...,...,...,...,...,...,...
5110,403.99723,0.47104,correct-go,DEV023,wave1,beta_0247.nii
5112,407.31460,0.56108,correct-go,DEV023,wave1,beta_0249.nii
5114,410.82016,0.54728,correct-go,DEV023,wave1,beta_0251.nii
5116,414.57572,0.49791,correct-go,DEV023,wave1,beta_0253.nii


There's actually no machine learning involved here. We're just calculating similarity between matrices.

In [5]:

def condition_resp_trans_func(X):
    return(X.trial_type)


In [6]:
test_train_set = pd.read_csv(train_test_markers_filepath)

with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)

dev_wtp_io_utils.check_BD_against_test_train_set(Brain_Data_allsubs,test_train_set) #ensure there's no test subjects in this dataset

checked for intersection and no intersection between the brain data and the subjects was found.
there were 15 subjects overlapping between the subjects marked for train data and the training dump file itself.


In [7]:
#need to clean the dataset before running similarity. not sure at what point we do this.
#first_subs_nifti = nil.image.clean_img(first_subs_nifti,detrend=False,standardize=True)
#time is along first axis
#https://github.com/nilearn/nilearn/blob/1607b52458c28953a87bbe6f42448b7b4e30a72f/nilearn/signal.py#L27
#nifti standardize gets the mean and std on axis zero (time axis) and standardizes
#brain_data standardize does the same https://nltools.org/_modules/nltools/data/brain_data.html#Brain_Data.standardize
#so we can replace nifti standardize with brain_data
bd_std = bd.standardize(method="zscore")

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


write up path for one subject. This will be encapsulated into a function or loop.

In [8]:
bd_std.X[['subject','trial_type']].value_counts()

subject  trial_type  
DEV009   correct-go      96
DEV019   correct-go      96
DEV018   correct-go      96
DEV014   correct-go      96
DEV012   correct-go      95
DEV017   correct-go      95
DEV023   correct-go      95
DEV021   correct-go      94
DEV010   correct-go      94
DEV013   correct-go      94
DEV016   correct-go      94
DEV006   correct-go      92
DEV015   correct-go      81
DEV005   correct-go      80
DEV022   correct-go      80
DEV015   correct-stop    20
DEV022   correct-stop    19
DEV006   correct-stop    18
DEV019   correct-stop    17
DEV018   correct-stop    15
DEV014   correct-stop    14
DEV016   correct-stop    14
DEV010   correct-stop    13
DEV023   correct-stop    13
DEV013   correct-stop    11
DEV021   correct-stop     8
DEV005   correct-stop     5
DEV017   correct-stop     4
dtype: int64

## test for one subject

In [104]:
#select subject
subj_i = 'DEV015' #np.unique(bd_std.X.subject)[0] #selected because it has the most correct-stop trials. we won't be able to do this every time.
subj_i_betas = bd_std.X.subject==subj_i
bd_subj_i = bd_std[subj_i_betas,]



Now we have a lot of trial-by-trial similarity measures to take; so we need to make this fast. Probably we need to do voxels in a small ROI or parcellate the brain. How do we do this?

In [105]:
bd_subj_i.shape()[0]*(bd_subj_i.shape()[0]-1)

10100

In [106]:
#run a representational similarity analysis. we need to simplify I think because we'd have 85*84=7140

In [114]:
bd_subj_i.X

,onset,duration,trial_type,subject,wave,beta
0,9.02224,0.68000,correct-go,DEV015,wave1,beta_0007.nii
1,12.52780,0.84117,correct-go,DEV015,wave1,beta_0009.nii
2,15.90975,0.47052,correct-go,DEV015,wave1,beta_0011.nii
3,18.35212,0.75253,correct-go,DEV015,wave1,beta_0013.nii
4,24.42644,0.55314,correct-go,DEV015,wave1,beta_0017.nii
...,...,...,...,...,...,...
96,395.98194,0.71608,correct-go,DEV015,wave1,beta_0241.nii
97,398.86111,0.66421,correct-go,DEV015,wave1,beta_0243.nii
98,401.49167,2.00556,correct-stop,DEV015,wave1,beta_0245.nii
99,403.99723,0.91995,correct-go,DEV015,wave1,beta_0247.nii


In [113]:
print(bd_subj_i.X.trial_type.value_counts())
print(", ".join(bd_subj_i.X.trial_type))

correct-go      81
correct-stop    20
Name: trial_type, dtype: int64
correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-stop, correct-stop, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, c

In [178]:
trial_subsample = range(20,50)

In [197]:
print(bd_subj_i.X.trial_type.value_counts())
print(", ".join(bd_subj_i.X.trial_type))

correct-go      81
correct-stop    20
Name: trial_type, dtype: int64
correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-stop, correct-stop, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-go, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-stop, correct-stop, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, correct-go, c

In [187]:
#just the first 5 images; 25 comparisons:
#API https://nltools.org/_modules/nltools/data/adjacency.html#Adjacency.distance_to_similarity
#https://nltools.org/api.html
#similarity_matrix = bd_subj_i[trial_subsample].distance(metric='correlation').distance_to_similarity()
similarity_matrix = bd_subj_i.distance(metric='correlation').distance_to_similarity()

In [188]:
#similarity_matrix.Y=bd_subj_i[trial_subsample].X
#similarity_matrix.labels=bd_subj_i[trial_subsample].X.trial_type.tolist()
similarity_matrix.labels=bd_subj_i.X.trial_type.tolist()

In [190]:
#similarity_matrix.plot()

In [207]:
#now we get average interclass correlation
similarity_matrix_arr = similarity_matrix.squareform()

#create dataframe to put it in
mat_length = similarity_matrix.square_shape()[0]
combo_count = int(mat_length*(mat_length-1)/2)
paired_list = pd.DataFrame({
  'interclass': np.empty(combo_count,bool),
    'similarity':np.empty(combo_count,float),
    'class_combo':np.empty(combo_count,str)
    
})
ij=0
for i in range(0,mat_length):
    for j in range(i+1,mat_length):
        interclass_i_j = similarity_matrix.labels[i]==similarity_matrix.labels[j]
        paired_list.loc[ij,'interclass']=interclass_i_j
        paired_list.loc[ij,'similarity']=similarity_matrix_arr[i,j]
        #create a label for this type of combination that is order-invariant
        if interclass_i_j:
            class_combo = similarity_matrix.labels[i]
        else:
            class_member_types = [similarity_matrix.labels[i],similarity_matrix.labels[j]]
            class_member_types.sort()
            class_combo = ", ".join(class_member_types)
        paired_list.loc[ij,'class_combo']=class_combo
        ij+=1
    

In [196]:
ttest_ind(paired_list[paired_list.interclass==True].cor,paired_list[paired_list.interclass==False].cor)

Ttest_indResult(statistic=-1.0205261981631928, pvalue=0.30752787597443937)

## Test whole-brain, within-subject, 15 subjects

Our goal here is to compare correlation within groups to correlation between groups. So we need to classify each correlation as within or between groups and then compare the averages.bd_subj_i

Now we can repeat the above per subject

In [109]:
def get_interclass_correlation(bd_subj_i,mask_b = None):
    if mask_b is None:
        similarity_matrix = bd_subj_i.distance(metric='correlation').distance_to_similarity()
    else:
        similarity_matrix = bd_subj_i.apply_mask(mask_b).distance(metric='correlation').distance_to_similarity()
    print(".",end="")
    similarity_matrix.labels=bd_subj_i.X.trial_type.tolist()
    
        #now we get average interclass correlation
    similarity_matrix_arr = similarity_matrix.squareform()

    print(".",end="")
    #create dataframe to put it in
    mat_length = similarity_matrix.square_shape()[0]
    combo_count = int(mat_length*(mat_length-1)/2)
    paired_list = pd.DataFrame({
      'interclass': np.empty(combo_count,bool),
        'similarity':np.empty(combo_count,float),
        'class_combo':np.empty(combo_count,str)

    })
    ij=0
    for i in range(0,mat_length):
        for j in range(i+1,mat_length):
            interclass_i_j = similarity_matrix.labels[i]==similarity_matrix.labels[j]
            paired_list.loc[ij,'interclass']=interclass_i_j
            paired_list.loc[ij,'similarity']=similarity_matrix_arr[i,j]
            #create a label for this type of combination that is order-invariant
            if interclass_i_j:
                class_combo = similarity_matrix.labels[i]
            else:
                class_member_types = [similarity_matrix.labels[i],similarity_matrix.labels[j]]
                class_member_types.sort()
                class_combo = ", ".join(class_member_types)
            paired_list.loc[ij,'class_combo']=class_combo
            ij+=1
            
    print(".",end="")
    
    return(paired_list)

In [41]:
list_of_paired_lists = []
for subj_i in np.unique(bd_std.X.subject):
    print(subj_i)
    subj_i_betas = bd_std.X.subject==subj_i
    bd_subj_i = bd_std[subj_i_betas,]
    
    unique_trial_types = len(np.unique(bd_std.X.trial_type))
    if unique_trial_types<2:
        print("only one trial type for " + subj_i + "; skipping.")
        continue
    
    paired_list = get_interclass_correlation(bd_subj_i)
    list_of_paired_lists = list_of_paired_lists + [paired_list]

DEV005
            similarity
interclass            
False         0.027057
True          0.022145
Ttest_indResult(statistic=-2.3663352670319404, pvalue=0.01801829634745891)
DEV006
            similarity
interclass            
False         0.083823
True          0.081466
Ttest_indResult(statistic=-0.9558332838350202, pvalue=0.3391950233996146)
DEV009


KeyboardInterrupt: 

,statistic,pvalue
0,-2.366335,0.018018
1,-0.955833,0.339195


In [100]:
results_list = []
for l in list_of_paired_lists:
    ttest_res = ttest_ind(l[l.interclass==True].similarity,l[l.interclass==False].similarity)
    combined_result_df = np.transpose(pd.DataFrame(l.groupby('class_combo').similarity.mean()))
    combined_result_df['statistic'] = ttest_res.statistic
    combined_result_df['pvalue'] = ttest_res.pvalue
   # print(combined_result_df)
    results_list = results_list + [combined_result_df]

pd.concat(results_list)

class_combo,correct-go,"correct-go, correct-stop",correct-stop,statistic,pvalue
similarity,0.022151,0.027057,0.020502,-2.366335,0.018018
similarity,0.080901,0.083823,0.096933,-0.955833,0.339195


OK, nothing here, but we've done whole-brain similarity! Now we need to think about similarity for specific regions an explore those interclass vs. intraclass differences.

In [ ]:
def iterate_through_subject

### Masking 

This is in luke's RSA guide, too. We can really siply take a k50_2mm mask and iterate through it to test similarity within each region. Unlike what Luke is doing in the tuorial with the mask, we'll be doing this across voxels within each region and not measuring relative activity in each area.



In [13]:
from nltools import expand_mask

In [25]:
mask = Brain_Data(ml_data_folderpath + "/Neurosynth Parcellation_0.nii.gz")
mask_names = pd.read_csv(ml_data_folderpath + "/parcellation_0_labels.csv")
mask_ex = expand_mask(mask)

In [26]:
mask_ex.X = mask_names

In [29]:
mask_names['Label']

0                                         anterior MPFC
1                              Fusiform/parahippocampus
2                                                 DMPFC
3                        Sensorimotor/Postcentral Gyrus
4                                                    V1
5            TPJ posterior supra marginal/angular gyrus
6                                         PCC/precuneus
7                                              Thalamus
8                                                   SMA
9                                      Precentral Gyrus
10                                       Cerebellum (VI
11                          Temporal Occipital Fusiform
12                                             Amygdala
13                              Superior Temporal Gyrus
14                                         white matter
15    TPJ anterior supra marginal gyrus/Parietal ope...
16                                           mid insula
17                                              

In [102]:
from IPython.display import display

In [112]:
correlations_by_mask = {}
#loop through masks
for mask_i in mask_ex:
    mask_name = mask_i.X[1]
    print(mask_name)
    mask_results_list = []
    #loop through subjects
    for subj_i in np.unique(bd_std.X.subject):
        print(subj_i,end=", ")
        
        #extract the data for this specific subject from the brain data
        subj_i_betas = bd_std.X.subject==subj_i
        bd_subj_i = bd_std[subj_i_betas,]

        #make sure we have enough data to analyze
        unique_trial_types = len(np.unique(bd_std.X.trial_type))
        if unique_trial_types<2:
            print("only one trial type for " + subj_i + "; skipping.")
            continue
            
        #get the interclass correlations. this does a LOT of work.
        paired_list = get_interclass_correlation(bd_subj_i,mask_i)
        l=paired_list

        #do statistical analysis on the results for this mask and subect
        ttest_res = ttest_ind(l[l.interclass==True].similarity,l[l.interclass==False].similarity)
        combined_result_df = np.transpose(pd.DataFrame(l.groupby('class_combo').similarity.mean()))
        combined_result_df['statistic'] = ttest_res.statistic
        combined_result_df['pvalue'] = ttest_res.pvalue
        
        mask_results_list = mask_results_list + [combined_result_df]
        
    mask_results_df = pd.concat(mask_results_list)
    display(mask_results_df)
    

    
    correlations_by_mask[mask_name] = mask_results_df

anterior MPFC
DEV005, ...DEV006, ...DEV009, ...DEV010, ...DEV012, ...DEV013, ...DEV014, ...DEV015, ...DEV016, ...DEV017, ...DEV018, ...DEV019, ...DEV021, ...DEV022, ...DEV023, ...

,correct-go,"correct-go, correct-stop",correct-stop,statistic,pvalue
similarity,0.045976,0.054992,0.055659,-2.990026,0.002809
similarity,0.097568,0.103251,0.120780,-1.664142,0.096136
similarity,0.119821,NaN,NaN,NaN,NaN
similarity,0.041187,0.038754,0.052227,1.497413,0.134341
similarity,0.076987,NaN,NaN,NaN,NaN
similarity,0.103743,0.109773,0.118772,-1.556129,0.119735
similarity,0.039133,0.040038,0.051614,-0.408339,0.683039
similarity,0.055128,0.057162,0.062929,-0.824648,0.409611
similarity,0.296630,0.300349,0.281094,-0.522111,0.601613
similarity,0.074591,0.089564,0.064732,-2.867694,0.004153


Fusiform/parahippocampus
DEV005, ...DEV006, ...DEV009, ...DEV010, ...DEV012, ...DEV013, ...DEV014, ...DEV015, ...DEV016, ...DEV017, ...DEV018, ...DEV019, ...DEV021, ...DEV022, ...DEV023, ...

,correct-go,"correct-go, correct-stop",correct-stop,statistic,pvalue
similarity,0.045807,0.055736,0.055809,-3.050152,0.002304
similarity,0.076183,0.075689,0.086415,0.355321,0.722361
similarity,0.062423,NaN,NaN,NaN,NaN
similarity,0.030098,0.029960,0.041319,0.215355,0.829498
similarity,0.058576,NaN,NaN,NaN,NaN
similarity,0.017540,0.018145,0.019662,-0.591807,0.554004
similarity,0.029736,0.031535,0.039081,-1.149510,0.250391
similarity,0.054124,0.056864,0.062523,-1.163435,0.244708
similarity,0.184975,0.177949,0.169755,1.308181,0.190864
similarity,0.078078,0.097808,0.072648,-3.637429,0.000278


DMPFC
DEV005, ...DEV006, ...DEV009, ...DEV010, ...DEV012, ...DEV013, ...DEV014, ...DEV015, ...DEV016, ...DEV017, ...DEV018, ...DEV019, ...DEV021, ...DEV022, ...DEV023, ...

...

,correct-go,"correct-go, correct-stop",correct-stop,statistic,pvalue
similarity,0.016517,0.021600,0.011089,-2.859995,4.261085e-03
similarity,0.085550,0.088815,0.099856,-1.060499,2.889604e-01
similarity,0.072440,NaN,NaN,NaN,NaN
similarity,0.057130,0.056482,0.072077,0.406534,6.843659e-01
similarity,0.060025,NaN,NaN,NaN,NaN
similarity,0.074818,0.075434,0.068231,-0.254280,7.992885e-01
similarity,0.032653,0.033695,0.043609,-0.550642,5.818995e-01
similarity,0.054281,0.056744,0.064059,-0.997227,3.187023e-01
similarity,0.176450,0.172877,0.160852,0.653361,5.135493e-01
similarity,0.072560,0.089243,0.062993,-3.215945,1.308693e-03


(Dataset incomplete)
DEV005, ...DEV006, ...DEV009, ...DEV010, ...DEV012, ...DEV013, ...DEV014, ...DEV015, ...DEV016, ...DEV017, ...DEV018, ...DEV019, ...DEV021, ...DEV022, ...DEV023, ...

,correct-go,"correct-go, correct-stop",correct-stop,statistic,pvalue
similarity,0.031191,0.039835,0.039043,-3.603186,0.000319
similarity,0.108220,0.110084,0.123556,-0.411095,0.681017
similarity,0.127402,NaN,NaN,NaN,NaN
similarity,0.042313,0.041525,0.050699,0.527318,0.597994
similarity,0.067012,NaN,NaN,NaN,NaN
similarity,0.054759,0.048995,0.050762,2.634325,0.008454
similarity,0.036724,0.040233,0.050491,-1.960365,0.049999
similarity,0.054405,0.057373,0.063689,-1.279981,0.200611
similarity,0.128864,0.130381,0.118741,-0.449363,0.653187
similarity,0.071371,0.086513,0.052289,-2.892011,0.003845


Left IFG
DEV005, ...DEV006, ..

KeyboardInterrupt: 